In [1]:
import datetime
import os
from OTXv2 import OTXv2
import time
import json 
import pandas as pd 
import requests
import urllib3
import json
import sys

In [2]:
pd.options.display.max_colwidth = 500
pd.set_option('display.max_columns', None)

In [3]:
import csv
import requests

json_URL = 'https://feodotracker.abuse.ch/downloads/ipblocklist_recommended.json'


with requests.Session() as s:
    download = s.get(json_URL)

    decoded_content = download.content.decode('utf-8','ignore')
    decoded_cleaned_content = decoded_content.replace('\n','')
    data = json.loads(decoded_cleaned_content)
    hostname_sslbl_df = pd.DataFrame.from_dict(pd.json_normalize(data))
hostname_sslbl_df

,ip_address,port,status,hostname,as_number,as_name,country,first_seen,last_online,malware
0,178.128.23.9,4125,online,None,14061,DIGITALOCEAN-ASN,SG,2021-05-16 19:49:33,2023-05-15,Dridex
1,104.248.178.90,4664,online,None,14061,DIGITALOCEAN-ASN,US,2021-07-28 16:01:42,2023-05-15,Dridex
2,192.99.150.39,7443,online,ns509018.ip-192-99-150.net,16276,OVH,CA,2021-08-27 04:48:12,2023-05-15,Dridex
3,128.199.232.159,6225,online,None,14061,DIGITALOCEAN-ASN,SG,2021-09-16 00:53:58,2023-05-15,Dridex
4,159.65.3.147,6225,online,None,14061,DIGITALOCEAN-ASN,SG,2021-09-17 11:44:00,2023-05-15,Dridex
...,...,...,...,...,...,...,...,...,...,...
253,86.178.33.63,2222,online,host86-178-33-63.range86-178.btcentralplus.com,2856,BT-UK-AS BTnet UK Regional network,GB,2023-05-15 14:44:30,2023-05-15,QakBot
254,200.109.16.12,2222,online,200.109.16-12.dyn.dsl.cantv.net,8048,"CANTV Servicios, Venezuela",VE,2023-05-15 14:53:39,2023-05-15,QakBot
255,92.98.159.9,2222,online,bba-92-98-159-9.alshamil.net.ae,5384,EMIRATES-INTERNET Emirates Internet,AE,2023-05-15 15:43:39,2023-05-15,QakBot
256,86.133.51.120,443,offline,host86-133-51-120.range86-133.btcentralplus.com,2856,BT-UK-AS BTnet UK Regional network,GB,2023-05-15 16:23:29,2023-05-15,QakBot


In [4]:
columns = ['ioc', 'ioc_type', 'threat_type', 'sha256_hash', 'sha3_384_hash',
       'sha1_hash', 'md5_hash', 'source', 'first_seen', 'last_seen',
       'file_name', 'file_type', 'reporter', 'threat',
       'origin_country', 'malware','tags', 'maliciousness', 'status', 'details', 'references']

In [5]:
hostname_sslbl_df.columns

Index(['ip_address', 'port', 'status', 'hostname', 'as_number', 'as_name',
       'country', 'first_seen', 'last_online', 'malware'],
      dtype='object')

In [6]:
export = pd.DataFrame()
export[columns] = ''

export

,ioc,ioc_type,threat_type,sha256_hash,sha3_384_hash,sha1_hash,md5_hash,source,first_seen,last_seen,file_name,file_type,reporter,threat,origin_country,malware,tags,maliciousness,status,details,references


In [7]:
export[[ 'first_seen', 'last_seen','file_name','origin_country','malware']] =  hostname_sslbl_df[['first_seen', 'last_online','as_name','country','malware']]
for i in range(len(export)):
    export['ioc'][i] = str(hostname_sslbl_df['ip_address'][i])+ ':' +str(hostname_sslbl_df['port'][i])

export['source'] = 'Feodo Tracker'
export['ioc_type'] = 'ip:port'
export['threat_type'] = 'botnet cc'

export

,ioc,ioc_type,threat_type,sha256_hash,sha3_384_hash,sha1_hash,md5_hash,source,first_seen,last_seen,file_name,file_type,reporter,threat,origin_country,malware,tags,maliciousness,status,details,references
0,178.128.23.9:4125,ip:port,botnet cc,NaN,NaN,NaN,NaN,SSL,2021-05-16 19:49:33,2023-05-15,DIGITALOCEAN-ASN,NaN,NaN,NaN,SG,Dridex,NaN,NaN,NaN,NaN,NaN
1,104.248.178.90:4664,ip:port,botnet cc,NaN,NaN,NaN,NaN,SSL,2021-07-28 16:01:42,2023-05-15,DIGITALOCEAN-ASN,NaN,NaN,NaN,US,Dridex,NaN,NaN,NaN,NaN,NaN
2,192.99.150.39:7443,ip:port,botnet cc,NaN,NaN,NaN,NaN,SSL,2021-08-27 04:48:12,2023-05-15,OVH,NaN,NaN,NaN,CA,Dridex,NaN,NaN,NaN,NaN,NaN
3,128.199.232.159:6225,ip:port,botnet cc,NaN,NaN,NaN,NaN,SSL,2021-09-16 00:53:58,2023-05-15,DIGITALOCEAN-ASN,NaN,NaN,NaN,SG,Dridex,NaN,NaN,NaN,NaN,NaN
4,159.65.3.147:6225,ip:port,botnet cc,NaN,NaN,NaN,NaN,SSL,2021-09-17 11:44:00,2023-05-15,DIGITALOCEAN-ASN,NaN,NaN,NaN,SG,Dridex,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,86.178.33.63:2222,ip:port,botnet cc,NaN,NaN,NaN,NaN,SSL,2023-05-15 14:44:30,2023-05-15,BT-UK-AS BTnet UK Regional network,NaN,NaN,NaN,GB,QakBot,NaN,NaN,NaN,NaN,NaN
254,200.109.16.12:2222,ip:port,botnet cc,NaN,NaN,NaN,NaN,SSL,2023-05-15 14:53:39,2023-05-15,"CANTV Servicios, Venezuela",NaN,NaN,NaN,VE,QakBot,NaN,NaN,NaN,NaN,NaN
255,92.98.159.9:2222,ip:port,botnet cc,NaN,NaN,NaN,NaN,SSL,2023-05-15 15:43:39,2023-05-15,EMIRATES-INTERNET Emirates Internet,NaN,NaN,NaN,AE,QakBot,NaN,NaN,NaN,NaN,NaN
256,86.133.51.120:443,ip:port,botnet cc,NaN,NaN,NaN,NaN,SSL,2023-05-15 16:23:29,2023-05-15,BT-UK-AS BTnet UK Regional network,NaN,NaN,NaN,GB,QakBot,NaN,NaN,NaN,NaN,NaN


In [8]:
now=datetime.datetime.utcnow().strftime("%Y%m%dT%H%M")
now

'20230515T1825'

In [9]:
export.to_csv(r'C:\Users\leona\OneDrive\Desktop\ThreatsData\threats\FeodoTracker_'+ now +'.csv')